In [1]:
import os
import sys
def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath + 'dataImport/')
from utils import exportsDataPath, writeHypnoDfParquet
import pandas as pd

import xml.etree.ElementTree as ET 

xmlFileName = exportsDataPath + "apple/17-9-24/export/apple_health_export/export.xml"
tree = ET.parse(xmlFileName)

root = tree.getroot() 

ColumnNames = ["startDate", "endDate", "creationDate", "sourceVersion", "value"]
parquetName = "appleWatchHypnoDf.parquet.gzip"

rType = "HKCategoryTypeIdentifierSleepAnalysis"
sourceName = "Abhik\'s Apple Watch"


some example data

 <Record type="HKCategoryTypeIdentifierSleepAnalysis" sourceName="Abhik’s Apple Watch" sourceVersion="9.0" creationDate="2022-10-01 09:03:33 -0700" startDate="2022-10-01 01:13:59 -0700" endDate="2022-10-01 01:29:59 -0700" value="HKCategoryValueSleepAnalysisAsleepDeep">
  <MetadataEntry key="HKTimeZone" value="America/Phoenix"/>
 </Record>
 <Record type="HKCategoryTypeIdentifierSleepAnalysis" sourceName="Abhik’s Apple Watch" sourceVersion="9.0" creationDate="2022-10-01 09:03:33 -0700" startDate="2022-10-01 01:29:59 -0700" endDate="2022-10-01 01:30:29 -0700" value="HKCategoryValueSleepAnalysisAwake">
  <MetadataEntry key="HKTimeZone" value="America/Phoenix"/>
 </Record>
 <Record type="HKCategoryTypeIdentifierSleepAnalysis" sourceName="Abhik’s Apple Watch" sourceVersion="9.0" creationDate="2022-10-01 09:03:33 -0700" startDate="2022-10-01 01:30:29 -0700" endDate="2022-10-01 01:43:29 -0700" value="HKCategoryValueSleepAnalysisAsleepCore">
  <MetadataEntry key="HKTimeZone" value="America/Phoenix"/>
 </Record>
 <Record type="HKCategoryTypeIdentifierSleepAnalysis" sourceName="Abhik’s Apple Watch" sourceVersion="9.0" creationDate="2022-10-01 09:03:33 -0700" startDate="2022-10-01 01:43:29 -0700" endDate="2022-10-01 01:50:59 -0700" value="HKCategoryValueSleepAnalysisAsleepREM">
  <MetadataEntry key="HKTimeZone" value="America/Phoenix"/>

In [2]:
print(len(root.findall("./Record")))

2275650


In [3]:
# rtypes = set([r.attrib['type'] for r in root.findall("./Record")])
# rtypes

In [4]:
[r.attrib['value'] for r in root.findall("./Record") if rType in r.attrib['type']].count('HKCategoryValueSleepAnalysisAsleepUnspecified')


2716

In [5]:
import pandas as pd

numRecords = 0
listRecords = []

sourceName = "Abhik’s Apple Watch"

val = 8

# for every element tagged Record
for r in root.findall("./Record"):
    # the data is in the attributes 
    record = r.attrib
    #if record["type"] == rType:
    #    print(record)
    if (record["type"] == rType and record["sourceName"] == sourceName): 
        numRecords += 1
        if numRecords % 100_000 == 0: print(numRecords)

        # the columns that I'm looking for are
        # sleep stage, start time, end time, source version
        # print(record["value"])
        val = -2
        if record["value"] == "HKCategoryValueSleepAnalysisAwake":
            val = 0
        elif record["value"] == "HKCategoryValueSleepAnalysisAsleepCore":
            val = 1
        elif record["value"] == "HKCategoryValueSleepAnalysisAsleepDeep":
            val = 2
        elif record["value"] == "HKCategoryValueSleepAnalysisAsleepREM":
            val = 3
        elif record["value"] == "HKCategoryValueSleepAnalysisAsleepUnspecified":
            val = 1
        # elif record["value"] == "HKCategoryValueSleepAnalysisInBed":
        #     val = 1

        if val != -2:
            row = [pd.to_datetime(record["startDate"]),
                   pd.to_datetime(record["endDate"]),
                   pd.to_datetime(record["creationDate"]), 
                   record["sourceVersion"],
                   val]
        
            listRecords.append(row)



print(len(listRecords))
print(numRecords)

21109
24649


In [6]:
set([x[4] for x in listRecords])

{0, 1, 2, 3}

In [7]:
from pytz import FixedOffset

tz = FixedOffset(-420)

# add in the no data segments
filledRecords = []
# for every row except the last
for ri in range(len(listRecords)-1):
    filledRecords.append(listRecords[ri])
    # if there is a gap between this rows end date and the next rows start date
    if listRecords[ri][1] < listRecords[ri+1][0]:
        filledRecords.append([
            listRecords[ri][1],
            listRecords[ri+1][0],
            pd.Timestamp.now(tz=tz),
            "synth",
            -1
        ])
print(len(filledRecords))

24153


In [8]:
print(len(filledRecords))

24153


In [9]:
# filledRecords[:100]

In [10]:
ColumnNames = ["startDate", "endDate", "creationDate", "sourceVersion", "value"]

AppleWatchHypnoDf = pd.DataFrame(columns=ColumnNames, data=filledRecords)

AppleWatchHypnoDf = AppleWatchHypnoDf.set_index("startDate")

In [11]:
AppleWatchHypnoDf

,endDate,creationDate,sourceVersion,value
startDate,,,,
2021-07-22 04:35:38-07:00,2021-07-22 06:12:38-07:00,2021-07-22 08:00:51-07:00,7.6,1
2021-07-22 06:12:38-07:00,2021-07-22 06:14:08-07:00,2024-09-17 13:13:31.663221-07:00,synth,-1
2021-07-22 06:14:08-07:00,2021-07-22 06:32:08-07:00,2021-07-22 08:00:51-07:00,7.6,1
2021-07-22 06:32:08-07:00,2021-07-22 06:33:38-07:00,2024-09-17 13:13:31.663259-07:00,synth,-1
2021-07-22 06:33:38-07:00,2021-07-22 08:00:08-07:00,2021-07-22 08:00:51-07:00,7.6,1
...,...,...,...,...
2024-09-17 07:23:21-07:00,2024-09-17 07:31:21-07:00,2024-09-17 10:37:33-07:00,10.6.1,3
2024-09-17 07:31:21-07:00,2024-09-17 07:31:51-07:00,2024-09-17 10:37:33-07:00,10.6.1,0
2024-09-17 07:31:51-07:00,2024-09-17 07:42:21-07:00,2024-09-17 10:37:33-07:00,10.6.1,1


In [12]:
AppleWatchHypnoDf.dtypes

endDate          datetime64[ns, pytz.FixedOffset(-420)]
creationDate     datetime64[ns, pytz.FixedOffset(-420)]
sourceVersion                                    object
value                                             int64
dtype: object

In [13]:
writeHypnoDfParquet('apple', AppleWatchHypnoDf)